In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Gradient Descent Algorithm with Regularizations (penalty term)

We can think of a function of two or more variables. We have a convex (one global minimum, no local minima) objective function (loss function), e.g. $$\large L(\beta_1, \beta_2)$$ which we try to minimize. In the case that there is no convex Loss function, the challenge is to fin the lowest local minimum (much harder problem).

We hope to share some ideas when $$\large L(\vec{\beta})$$ is convex in th evector $\vec{\beta}$. In particular, we are interested in the situation where $L$ is the sum of squared errors plus a penalty (regularization) term, such as: 
$$\large L(\vec{\beta}): = \displaystyle\sum_{i=1}^{n} (y_i-x_i\cdot \vec{\beta})^2 + P_{\alpha}(\beta)$$

For an example of P, take elastic net: $$P_{\alpha}(\beta) = \alpha \cdot (\lambda\cdot \sum |\beta_j| + (1-\lambda))\cdot \sum (\beta_j^2)$$
In sklearn, $\frac{\lambda}{1-\lambda}$ is called the L1 ratio

Notice that this yields quadratics in the components of beta, which means this is a convex loss function (high dimensional parabola)

To reduce this problem to a one dimensional problem, we think of perturbing an ideal $\vec{\beta}$. We want to know how to update $\vec{\beta}$ so that we make the best progress in minimizing $L$. Think of updating $\vec{\beta}$ by going $t$ units in a direction $\vec{v}$. So we ask, what is the vector $\vec{v}$ that gives the best progress?

We consider some $$g(t) :=L(\vec{\beta} + t \cdot \vec{v})$$
Notice that, from the chain rule, the derivative of $g(t)$ is $$\large g'(t) = \nabla L \cdot \vec{v}$$ so for the most effective step of sie $t$, we want the direction vector $$\large \vec{v} = - \nabla L$$

So the gradient in the case of elastic net would be

$$\large -\nabla L(\vec{\beta}): = - \frac{\partial}{\partial\beta} \displaystyle\sum_{i=1}^{n} (y_i-x_i\cdot \vec{\beta})^2 - \alpha\frac{\partial}{\partial\beta}(\lambda\cdot \sum |\beta_j| + (1-\lambda))\cdot \sum (\beta_j^2)$$ 

which becomes

$$  - \frac{\partial L}{\partial\beta} = -2\displaystyle\sum_{i=1}^{n}(y_i-\sum_{j=1}^{p} x_{ij} \cdot \beta_j)\cdot(x_{ij}) + \alpha\lambda sign(\beta_j)) $$


<font color='red' size=5pt> This works well if $L$ is convex in $\vec{\beta}$ and the only issue would be when ther eis a very shallow basin of the minimum for $L$.

### Below we use this gradient descent algorithm with multiple regularizers

In [2]:
def gradient_mse(beta, x, y, alpha, l): # we defined a function that computes the gradient of the objective function
    n = len(y) # the number of observations
    y_hat = x.dot(beta).flatten()
    error = (y - y_hat)
    mse = (1.0 /n) * np.sum(np.square(error)) + alpha*(l*np.sum(np.abs(beta))+(1-l)*np.sum(beta**2)) #addid bit (penalty) is for ridge regression(?)
    gradient = -(2.0 /n) * error.dot(x) + 2*(1-l)*beta*alpha + alpha*l*np.sign(beta) #Gradeint is the derivative of MSE in beta and in the error dimensions
    return gradient, mse

In [3]:
data = pd.read_csv('Data/Concrete_Data.csv')
data

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [4]:
X = data[data.columns[0:7]]
y = data[data.columns[-1]].values

In [5]:
scale = StandardScaler()
X_scaled = scale.fit_transform(X)
X_scaled.shape
#y.reshape(-1,1).shape

(1030, 7)

In [6]:
beta = np.array((-60, -60,-60, -60,-60, -60, -60)).reshape(-1,1) # a very imprecise guess to initialize the coefficients
lr = .05 # the learning rate
tolerance = 1e-8 #if progress drops below this, stop
alpha = 0.01
 
old_beta = []
mse = []
print(beta)

[[-60]
 [-60]
 [-60]
 [-60]
 [-60]
 [-60]
 [-60]]


In [7]:
# Perform Gradient Descent
iterations = 1
for i in range(500):
    gradient, mse_temp = gradient_mse(beta, X_scaled, y.reshape(-1,1), alpha, l=lr)
    new_beta = beta - lr * gradient # here we update the coefficients in the direction of the negative gradient
 
    # Print error every 10 iterations
    if iterations % 10 == 0:
        print("Iteration: %d - Mean Squarred Error: %.4f" % (iterations, mse_temp))
        old_beta.append(new_beta)
        mse.append(mse_temp)
 
    # Stopping Condition
    if np.sum(abs(new_beta - beta)) < tolerance:
        print('The Gradient Descent Algorithm has converged')
        break
 
    iterations += 1
    beta = new_beta
 
print('w =', w)

ValueError: operands could not be broadcast together with shapes (1030,7) (7,1) 

In [8]:
X_scaled[0]

array([ 2.47791487, -0.85688789, -0.84714393, -0.91676439, -0.62044832,
        0.86315424, -1.21767004])

In [9]:
0.05/(1-0.05)

0.052631578947368425